In [34]:
import openai
import langchain
import pinecone 
# from langchain_pinecone import Pinecone
from langchain.document_loaders import PyPDFDirectoryLoader # for loading pdf
from langchain.text_splitter import RecursiveCharacterTextSplitter  #to tokenize into chunks
from  langchain.embeddings.openai import OpenAIEmbeddings # chunks to vectores
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp

from dotenv import load_dotenv
load_dotenv(override=True)
import os


In [35]:

## Read document
def read_doc(diretory):
    file_loader = PyPDFDirectoryLoader(diretory)
    print(file_loader)
    documents= file_loader.load()
    return documents

docs=read_doc("documents/")
print('len',len(docs))
print(type(docs))
# for doc in docs:
#     print(doc)


len 58
<class 'list'>


In [36]:

## Divide the docs into chunks
### https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#
def chunk_data(docs,chunk_size,chunk_overlap):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return doc 

chunkData = chunk_data(docs=docs, chunk_overlap=50, chunk_size=800)
# chunkData


In [37]:

print("Type: ",type(chunkData), "Len:", len(chunkData))
print(chunkData[2])



Type:  <class 'list'> Len: 141
page_content=' Rationalisation  
 Others  
 Personal Income Tax  
  
Annexures 35 
 Annexure to Part B of the Budget Speech 2023-24 
i. Amendments relating to Direct Taxes 
ii. Amendments relating to Indirect Taxes' metadata={'source': 'documents/budget_speech.pdf', 'page': 2, 'page_label': '3'}


In [38]:
## Embedding Technique Of OPENAI
embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
embeddings

vectors=embeddings.embed_query("How are you?")

In [39]:

#Connect to Pinecone
#Connect to Pinecone
pc = pinecone.Pinecone(api_key=os.environ['PINECONE_API_KEY'])  # Use environment variable
# Connect to your index

pinceConeIndex=Pinecone.from_documents(documents=chunkData , embedding=embeddings, index_name="langchain")
print(pinceConeIndex)

In [40]:

# Initialize the LLM with your API key
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.5,
    openai_api_key=os.environ.get("OPENAI_API_KEY")
)

# Load the QA chain using the 'stuff' chain type
chain = load_qa_chain(llm, chain_type="stuff")
# chain=load_qa_chain(llm,chain_type="stuff")



In [41]:
# Using Cosine similarity retrive mathcing results from vectorDB

def retrive_query(query,k=2):
    matching_results=pinceConeIndex.similarity_search(query,k=k)
    return matching_results
# Search Answers from Vector DB
def retrieve_answers(query):
    doc_search=retrive_query(query=query)
    print(doc_search)
    response=chain.run(input_documents=doc_search, question=query)
    return response
    


In [42]:
query="what areas are going to get improved in the coming budget"
print(retrieve_answers(query))

[Document(metadata={'page': 13.0, 'page_label': '14', 'source': 'documents/budget_speech.pdf'}, page_content='improved access to education, health and nutrition, road and telecom \nconnectivity, and sustainable livelihood opportunities. An amount  \nof ` 15,000 crore will be made available to implement the Mission in the \nnext three years under the Development Action Plan for the Scheduled \nTribes.  \nEklavya Model Residential Schools \n38. In the next three years, centre will recruit 38,800 teachers and \nsupport staff for the 740 Eklavya Model Residential Schools, serving 3.5 lakh \ntribal students. \nWater for Drought Prone Region \n39. In the drought prone central region of Karnataka, central assistance \nof ` 5,300 crore will be given to Upper Bhadra Project to provide \nsustainable micro irrigation and filling up of surface tanks for drinking \nwater.  \nPM Awas Yojana'), Document(metadata={'page': 13.0, 'page_label': '14', 'source': 'documents/budget_speech.pdf'}, page_content